### Classification Problem Using Artificial Neural Netwrok
### In this notebook we will develop a deep learning model to predict the probability of customer churn for a bank. The features that we will use to predict this include geographic location, age, credit score,current balance, length of their tenure, etc.

### Importing the libraries

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf

### Part 1 - Data Preprocessing
###  Here we first divide the data into feature set and the target label (i,e is whether the customer has left the bank or not)

### Importing the dataset

In [ ]:
url='https://raw.githubusercontent.com/ehsanece/ML_Data_Drive/main/Churn_Modelling.csv'
dataset =pd.read_csv(url,error_bad_lines=False)
#dataset = pd.read_csv('Churn_Modelling.csv')
print(dataset.head(10))
X = dataset.iloc[:, 3:-1].values
y = dataset.iloc[:, -1].values

   RowNumber  CustomerId   Surname  ...  IsActiveMember EstimatedSalary Exited
0          1    15634602  Hargrave  ...               1       101348.88      1
1          2    15647311      Hill  ...               1       112542.58      0
2          3    15619304      Onio  ...               0       113931.57      1
3          4    15701354      Boni  ...               0        93826.63      0
4          5    15737888  Mitchell  ...               1        79084.10      0
5          6    15574012       Chu  ...               0       149756.71      1
6          7    15592531  Bartlett  ...               1        10062.80      0
7          8    15656148    Obinna  ...               0       119346.88      1
8          9    15792365        He  ...               1        74940.50      0
9         10    15592389        H?  ...               1        71725.73      0

[10 rows x 14 columns]


### Encoding categorical data
### We use label enoding as the gender has only two values

###Label Encoding the "Gender" column

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])

# One Hot Encoding the "Geography" column
### The geography has many categorical values, so we use one hot encoding

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

In [ ]:
print(X)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


### Splitting the dataset into the Training set and Test set

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

### Feature Scaling

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## Part 2 - Building the ANN

### Initializing the ANN

In [ ]:
ann = tf.keras.models.Sequential()

### Adding the input layer and the first hidden layer
### We use the Rectifier Linear Unit for the activation of the neurons

In [ ]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

### Adding the second hidden layer

In [ ]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

### Adding the output layer
### We use the  sigmoid acitivation as we want to predict the probablity

In [ ]:
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

### Part 3 - Training the ANN

### Compiling the ANN

In [ ]:
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the ANN on the Training set

In [ ]:
ann.fit(X_train, y_train, batch_size = 32, epochs = 100)

Epoch 1/100
250/250 [==============================] - 1s 1ms/step - loss: 0.5867 - accuracy: 0.7700
Epoch 2/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4620 - accuracy: 0.7960
Epoch 3/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4412 - accuracy: 0.7960
Epoch 4/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4317 - accuracy: 0.7960
Epoch 5/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4247 - accuracy: 0.7995
Epoch 6/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4172 - accuracy: 0.8198
Epoch 7/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4093 - accuracy: 0.8254
Epoch 8/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4029 - accuracy: 0.8254
Epoch 9/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3980 - accuracy: 0.8282
Epoch 10/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3943 - accura

### Part 4 - Making the predictions and evaluating the model

### For a single entry

In [ ]:
print(ann.predict(sc.transform([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])) > 0.5)


[[ True]]


Here we evaluate our model on the test set

In [ ]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)
#print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

### Evaluating the Performance of our model

### Making the Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

cm = confusion_matrix(y_test, y_pred)
print("Confution Matrix\n")
print(cm)
print("\n")
print("Accuracy Score\n")
print(accuracy_score(y_test, y_pred))
print("\n")
print("Classification Report\n")
cr=classification_report(y_test, y_pred)
print(cr)


Confution Matrix

[[1510   85]
 [ 200  205]]


Accuracy Score

0.8575


Classification Report

              precision    recall  f1-score   support

           0       0.88      0.95      0.91      1595
           1       0.71      0.51      0.59       405

    accuracy                           0.86      2000
   macro avg       0.79      0.73      0.75      2000
weighted avg       0.85      0.86      0.85      2000

